In [1]:
!pip uninstall -y protobuf tensorflow-metadata
!pip install protobuf==3.20.3 tensorflow-metadata==1.13.1


Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
Found existing installation: tensorflow-metadata 1.17.2
Uninstalling tensorflow-metadata-1.17.2:
  Successfully uninstalled tensorflow-metadata-1.17.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.5 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is inco

In [2]:
import os

print(os.listdir("/kaggle/input/mepe-text-emotion-hf"))


['__huggingface_repos__.json', 'mepe']


In [3]:
import os

BASE = "/kaggle/input/mepe-text-emotion-hf/mepe/models/text_emotion_hf"

print(os.listdir(BASE))


['config.json', 'tokenizer.json', 'tf_model.h5', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt']


we got the output of the phase1: text_emotion.ipynb, by inputting it to the current notebook as a datset after exporting it through the phase1: text_emotion.ipynb as datset

# imports

In [4]:
import tensorflow as tf
import numpy as np
import os


2025-12-15 02:59:13.764788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765767554.041308      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765767554.117309      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Paths & Config

In [5]:


TEXT_DIM = 768
FACE_DIM = 256
FUSION_DIM = 512
NUM_EMOTIONS = 28   # text emotion space


# Load pretrained encoders (FROZEN)
Text encoder (DistilBERT backbone only)

In [6]:
from transformers import AutoTokenizer, TFDistilBertModel

HF_TEXT_DIR = "/kaggle/input/mepe-text-emotion-hf/mepe/models/text_emotion_hf"

tokenizer = AutoTokenizer.from_pretrained(
    HF_TEXT_DIR,
    local_files_only=True
)

text_encoder = TFDistilBertModel.from_pretrained(
    HF_TEXT_DIR,
    local_files_only=True
)

print("HF model accessible")


2025-12-15 02:59:50.024334: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at /kaggle/input/mepe-text-emotion-hf/mepe/models/text_emotion_hf were not used when initializing TFDistilBertModel: ['classifier', 'dropout_19', 'pre_classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClass

HF model accessible


# Face embedding model

In [7]:
face_encoder = tf.keras.models.load_model(FACE_EMB_DIR)
face_encoder.trainable = False


NameError: name 'FACE_EMB_DIR' is not defined

# Gated Fusion Layer (CORE LOGIC)

In [ ]:
class GatedFusion(tf.keras.layers.Layer):
    def __init__(self, fusion_dim):
        super().__init__()
        self.text_proj = tf.keras.layers.Dense(fusion_dim)
        self.face_proj = tf.keras.layers.Dense(fusion_dim)
        self.gate = tf.keras.layers.Dense(fusion_dim, activation="sigmoid")

    def call(self, text_emb, face_emb):
        t = self.text_proj(text_emb)
        f = self.face_proj(face_emb)
        g = self.gate(tf.concat([t, f], axis=-1))
        return g * t + (1 - g) * f


# Build Fusion Model

In [ ]:
# Inputs
text_input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
text_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
face_input = tf.keras.Input(shape=(256,), name="face_embedding")

# Text encoding
text_out = text_encoder(
    input_ids=text_input_ids,
    attention_mask=text_mask
).last_hidden_state[:, 0, :]   # CLS token

# Fusion
fusion_layer = GatedFusion(FUSION_DIM)
fusion_emb = fusion_layer(text_out, face_input)

# Output head
x = tf.keras.layers.Dense(256, activation="relu")(fusion_emb)
logits = tf.keras.layers.Dense(NUM_EMOTIONS, name="emotion_logits")(x)

fusion_model = tf.keras.Model(
    inputs=[text_input_ids, text_mask, face_input],
    outputs=logits
)

fusion_model.summary()


# compile

In [ ]:
fusion_model.compile(
    optimizer=tf.keras.optimizers.Adam(2e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


# Dummy sanity test 

Before real training, verify forward pass.

In [ ]:
dummy_text = tokenizer(
    ["I feel very anxious today"],
    return_tensors="tf",
    padding=True
)

dummy_face = tf.random.normal((1, 256))

out = fusion_model([
    dummy_text["input_ids"],
    dummy_text["attention_mask"],
    dummy_face
])

out.shape


# Training strategy (READ THIS)

At Phase 4:

We do NOT retrain encoders

We train only fusion layers

This avoids:

instability

massive compute

overfitting

We will train on paired (text, face) samples later
(real or synthetic).

For now, architecture correctness is the goal.

# Save fusion model

In [ ]:
os.makedirs(FUSION_DIR, exist_ok=True)
fusion_model.save(FUSION_DIR)

print("Fusion model saved.")


# Persona embedding extractor

In [ ]:
persona_model = tf.keras.Model(
    inputs=fusion_model.inputs,
    outputs=fusion_emb
)

persona_model.summary()
